In [78]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow_probability import distributions as dists
import tensorflow.keras.layers as kl

from rl_agents.env_utils import rollouts_generator, get_adv_vtarg
from rl_agents.ppo.policy import Actor, Critic
from rl_agents.ppo.agent import PPO_Agent

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

tf.random.set_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
env = gym.make('Pendulum-v0')
is_continuous = isinstance(env.action_space, gym.spaces.Box)
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape

In [80]:
actor = Actor(obs_dim, act_dim, is_continuous)
critic = Critic(obs_dim)
vero = PPO_Agent(actor, critic)

In [81]:
generator = rollouts_generator(vero, env, horizon=2048)
rollout = generator.__next__()

TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])


TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


...

TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])


TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])


TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
T

TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])
TensorShape([1, 1])
TensorShape([1, 1])
TensorShape([1, 3])
TensorShape([1, 32])
TensorShape([1, 32])


In [82]:
advantage, target_value = get_adv_vtarg(rollout, lam=0.95, gamma=0.99)

In [84]:
vero.run_epoch(rollout['ob'], rollout['ac'], rollout['log_probs'], target_value, advantage, epochs=2)

(2048, 3)
(64, 3)
TensorShape([1, 64, 3])
TensorShape([1, 64, 32])
TensorShape([1, 64, 32])
TensorShape([1, 64, 1])
TensorShape([1, 64, 1])
tf.Tensor(
[[[-1.01220427]
  [-1.23525365]
  [-1.85962878]
  [-0.51570951]
  [-0.47730788]
  [-1.17664472]
  [-0.65195884]
  [-0.67754954]
  [-1.50740896]
  [-0.63371517]
  [-0.39232662]
  [-0.61800156]
  [-0.75498964]
  [-0.48372643]
  [-0.65919747]
  [-2.40100035]
  [-0.48236372]
  [-0.65593636]
  [-0.3889289 ]
  [-0.38903896]
  [-1.24826061]
  [-0.45191375]
  [-0.39045416]
  [-1.27019323]
  [-0.42725812]
  [-0.48479661]
  [-0.53951055]
  [-0.69181139]
  [-0.38875992]
  [-1.23905329]
  [-1.43430216]
  [-0.67034795]
  [-0.61860256]
  [-1.99247911]
  [-0.43276235]
  [-0.41608112]
  [-0.41989473]
  [-2.24970738]
  [-0.39429424]
  [-0.58581113]
  [-0.39279892]
  [-0.64931647]
  [-0.42484938]
  [-0.40296479]
  [-0.41518081]
  [-0.39838993]
  [-0.83289614]
  [-0.39000512]
  [-0.64365513]
  [-0.401012  ]
  [-0.49019886]
  [-0.54970339]
  [-0.42217644]
 

In [56]:
obs = env.reset()
ac, v, lp = vero.act_stochastic(obs)
print(lp)
l = vero.get_distributions(obs)
print(l.log_prob(ac))
print(l.entropy())

tf.Tensor([-1.01404747], shape=(1,), dtype=float64)
tf.Tensor([[-1.01404747]], shape=(1, 1), dtype=float64)
tf.Tensor([[0.88893853]], shape=(1, 1), dtype=float64)


In [13]:
ac = env.action_space.sample()
print(ac.dtype)

float32


In [18]:
print(np.float64)

<class 'numpy.float64'>


In [31]:
import tensorflow_probability as tfp

In [32]:
tfd = tfp.distributions


In [34]:
dist = tfd.Normal(loc=np.array([0., 1, 2, 0.5]), scale=np.array([0.02, 0.09, 0.1, 1]))

In [36]:
dist.sample()


<tf.Tensor: id=525722, shape=(4,), dtype=float64, numpy=array([-0.0065997 ,  0.99706162,  1.96786689, -0.39310536])>

In [73]:
obs_dim

(3,)